In [17]:
from dscigametrics.compute_metrics import compute_metrics
import pandas as pd

In [18]:
data = pd.read_csv('../tests/ga_metrics_test_data.csv')

### `compute_metrics` - Calculate four key metrics from GA data

Google analytics provides a lot of information, so much that it can be dizzying looking at the raw spreadsheet. Fortunately we can start to assess the performance of a marketing campaign using just five columns, shown below.

In [19]:
data[['date',
'trafficSource.adwordsClickInfo.campaignId',
'totals.newVisits',
'totals.transactions',
'totals.transactionRevenue'
]].head()

,date,trafficSource.adwordsClickInfo.campaignId,totals.newVisits,totals.transactions,totals.transactionRevenue
0,20220801,219011657,1.0,NaN,NaN
1,20220802,219011657,NaN,NaN,NaN
2,20220803,219011657,1.0,NaN,NaN
3,20220804,219011657,1.0,NaN,NaN
4,20220805,219011657,1.0,NaN,NaN


Each row represents an event where someone clicked on an add. The first column is the date of that event, then we have the unique ID of the campaign that the add belonged to, then a column which records whether the visit was from someone new (1.0 means new), then whether a purchase was made (1.0 indicates a purchase), and the last column records the revenue made on that purchase. 

The function `compute_metrics` uses the first two columns to filter based on your specifications: you are to provide the ID of the campaign that you're interested in, and specify the date range that you want to see data from.

`compute_metrics` then uses the last three columns to compute four key metrics:

**Conversion rate:** this is the percentage of events where a purchase was made. Note that this is a percentage of the total number of clicks rather than individuals: the same person could click on an add 100 times but only make a purchase following the last one; the conversion rate in this case would be 1%. This tells you how successful the add is in actually selling products.

**New to return rate:** this is the percentage of clicks that are made by someone who hasn't clicked on the add before. In the last example, this woud also be 1% because only one of the 200 clicks was from a new person. This gives a sense of both how many individuals are seeing the add and whether they're clicking on the add multiple times.

**Total transaction revenue:** this is the total amount of money made from all purchases made after clicking on an add. 

**Average transaction revenue:** this is the total amount of money made but divided over the total number of times someone purchased something after clicking on an add. These last two metrics give you a sense of whether revenue is coming from a few small purchases or a few large ones.

Let's try it out! We'll collect data for the add campaign with id 123851219, starting on the 1st of August 2022, up to *and including* the 31st of August, 2022.

In [20]:
metric_dict = compute_metrics(data, 123851219, 20220801, 20220831)

conversion rate: 0.116 
new to return rate: 0.88 
total transaction revenue: $14548.0 
average transaction revenue: $501.6551724137931


It's as easy as that! Take note of a few things here: the function prints out a few lines, but there's no need to copy this down somewhere. If you assign the output of the function to an object like we've done here  with `metric_dict`, the results will be stored there as a dictionary. You can access individual metrics from it like this:

In [21]:
conversion_rate = metric_dict['conversion rate']

print(conversion_rate)

0.116


Also notice that the function does not round its outputs at all. This is to make sure you are in control of precision in case this matters for subsequent calculations, but it also means you'll probably want to round off the results before presenting them elsewhere.

In [22]:
metric_dict['average transaction revenue'].round(2)

501.66